## Heart Rate Processing and Baseline Correction 
Script by Nadu Barbashova 

This Script is also on Github in my Psychophysiology Folder: 
https://github.com/n-barbashova/psychophysiology/tree/master/HeartRate 

Neurokit Documentation: 


### About this script: 
My Experiment: 
Participants wait 60 seconds for stimuli (shocks or stimulations). In this time they watch a 60 second coundown. For 30 of these seconds they complete the flanker task and for the other 30 seconds they just watch the countdown. The order depends on which countdown it is. There are 36 countdowns, divided into 9 runs. 

The goal is to see how the heartrate is affected by the condition (is this a threat countdown (leading to shock) or a no-threat countdown (leading to stim)). I want to also see how heart rate is affected when it's the flanker task epoch vs the countdown epoch. 


### What this script does:
This script takes the pre-processed ECG data and further processes it using functions from Neurokit. It cleans the heart rate, finds event codes and using the event codes, labels which condition this is (shock or stim), which interval it is (flanker or countdown). In addition to processing the heart rate it also does one second heart rate baseline correction, comparing the heart rate in the given interval to the heart rate in the one second before the countdown began. 


### Input required: 


### Output from this script: 
This script will output a new file for each input file. 


In [11]:

# Load NeuroKit and other useful packages (make sure they are installed)
import neurokit2 as nk
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# subject ID's to loop through 
IDs = ["49", "50", "51", "52", "54", "55", "56", "57", "58", "61", "62", "63", "65", "67",
          "68", "69", "70", "72", "73", "74", "76", "78", "81", "82", "84",
       "85", "86", "87", "88", "89", "91", "93", "98", "99", "100"]

# runs to loop through 
runs = [ 1, 2, 3, 4, 5, 6, 7, 8, 9]

#input_dir = 
outdir = "/Users/nadezhdabarbashova/Desktop/fmcc_timing/neurokit/"

# plot the heart rate - it creates pretty funny plots where you can see literally evety heart beat.  
# Not the most useful but nice to double check the data for noise or disrupted signal 
figs_dir = "/Users/nadezhdabarbashova/Desktop/fmcc_timing/HR/figs/"


In [ ]:
## Heart Rate Processing and Baseline Correction 
### updated to have the events labelled

# Define Directories
flanker_start_events = [3, 7, 11, 15]  # Define relevant event codes


for ID in IDs:
    for run in runs:
        try:
            print(f"Processing {ID} Run {run}...")

            # Load data
            fn = f"/Users/nadezhdabarbashova/Desktop/fmcc_timing/HR/{ID}_run{run}.txt"
            data = pd.read_csv(fn, sep="\t", header=None)
            data.columns = ["timepoint", "ECG", "EVENT"]

            # Process ECG data (clean, detect peaks, compute HR)
            signals, info = nk.ecg_process(data["ECG"], sampling_rate=100)

            # Merge processed ECG data with original event data
            datasignal = pd.concat([signals, data[["timepoint", "EVENT"]]], axis=1)

            # --------------------------------------
            # Add Condition Labels to `datasignal`
            # --------------------------------------
            
            datasignal['event_chunk_countdown'] = 0
            datasignal['event_chunk_flanker'] = 0
            datasignal['threat_type'] = ""
            datasignal['distance_type'] = ""
            datasignal['event_type'] = ""

            event_pairs_countdown = [(1, 2), (5, 6), (9, 10), (13, 14)]
            event_pairs_flanker = [(3, 4), (7, 8), (11, 12), (15, 16)]

            for start, end in event_pairs_countdown:
                if (datasignal['EVENT'] == start).any() and (datasignal['EVENT'] == end).any():
                    start_idx = datasignal.index[datasignal['EVENT'] == start][0]
                    end_idx = datasignal.index[datasignal['EVENT'] == end][0]
                    datasignal.loc[start_idx:end_idx + 1, 'event_chunk_countdown'] = start

            for start, end in event_pairs_flanker:
                if (datasignal['EVENT'] == start).any() and (datasignal['EVENT'] == end).any():
                    start_idx = datasignal.index[datasignal['EVENT'] == start][0]
                    end_idx = datasignal.index[datasignal['EVENT'] == end][0]
                    datasignal.loc[start_idx:end_idx, 'event_chunk_flanker'] = start

            # Label conditions
            datasignal.loc[datasignal['event_chunk_countdown'] == 1, ['threat_type', 'distance_type', 'event_type']] = ['shock', 'distal', 'distal shock countdown']
            datasignal.loc[datasignal['event_chunk_flanker'] == 3, ['threat_type', 'distance_type', 'event_type']] = ['shock', 'distal', 'distal shock flanker']
            datasignal.loc[datasignal['event_chunk_countdown'] == 5, ['threat_type', 'distance_type', 'event_type']] = ['shock', 'proximal', 'proximal shock countdown']
            datasignal.loc[datasignal['event_chunk_flanker'] == 7, ['threat_type', 'distance_type', 'event_type']] = ['shock', 'proximal', 'proximal shock flanker']
            datasignal.loc[datasignal['event_chunk_countdown'] == 9, ['threat_type', 'distance_type', 'event_type']] = ['stim', 'distal', 'distal stim countdown']
            datasignal.loc[datasignal['event_chunk_flanker'] == 11, ['threat_type', 'distance_type', 'event_type']] = ['stim', 'distal', 'distal stim flanker']
            datasignal.loc[datasignal['event_chunk_countdown'] == 13, ['threat_type', 'distance_type', 'event_type']] = ['stim', 'proximal', 'proximal stim countdown']
            datasignal.loc[datasignal['event_chunk_flanker'] == 15, ['threat_type', 'distance_type', 'event_type']] = ['stim', 'proximal', 'proximal stim flanker']

            # Add 'end' condition labels
            datasignal.loc[datasignal['EVENT'] == 2, 'event_type'] = 'distal shock countdown end'
            datasignal.loc[datasignal['EVENT'] == 6, 'event_type'] = 'proximal shock countdown end'
            datasignal.loc[datasignal['EVENT'] == 10, 'event_type'] = 'distal stim countdown end'
            datasignal.loc[datasignal['EVENT'] == 14, 'event_type'] = 'proximal stim countdown end'

            # Remove rows where event_chunk_flanker == 0

        
            # Save full dataset with ECG metrics
            save_filename = os.path.join(outdir, f"{ID}_EKG_ECode_HeartRate{run}.csv")
            
 
            datasignal.to_csv(save_filename, index=False)
            print(f"✅ Saved full ECG dataset: {save_filename}")

            # Identify Flanker Start Events
            flanker_events_df = datasignal[datasignal["EVENT"].isin(flanker_start_events)]
            event_indices = flanker_events_df.index.tolist()

            if not event_indices:
                print(f"⚠️ No flanker start events found for {ID} run {run}. Skipping epoch creation.")
                continue

            # Create Epochs (using processed ECG data) - this is a dictionary 
            epochs = nk.epochs_create(
                data = datasignal,  # Use full processed dataset
                events = event_indices,
                sampling_rate=100, 
                epochs_start=-1, # start at -1 second before the even 
                epochs_end=30
            )

            if not epochs:
                #print(f"No epochs created for {ID} run {run}.")
                continue

            # Save Each Epoch Separately
            for event_label, epoch_df in epochs.items():
                
                ### NEW CODE: Baseline Correction
                try:
                    # Compute baseline (ECG_Rate) from -1 second to event onset (first 100 rows)
                    ecg_baseline = epoch_df["ECG_Rate"].iloc[:100].mean()  

                    # Compute the mean ECG_Rate for the 30 seconds after event onset (next 3000 rows)
                    ecg_mean = epoch_df["ECG_Rate"].iloc[100:].mean()  

                    # Create a new column in epoch_df for the corrected ECG rate
                    epoch_df["ECG_Baseline_Corrected"] = epoch_df["ECG_Rate"] - ecg_baseline  
                    print("Baseline Correction Done")

                except Exception as e:
                    print(f"⚠️ Error in baseline correction for {ID} run {run}, event {event_label}: {e}")
                    epoch_df["ECG_Rate_Corrected"] = epoch_df["ECG_Rate"]  # If error, keep original values

                epoch_df = epoch_df[epoch_df["event_chunk_flanker"] != 0]
                epoch_df = epoch_df.rename(columns={"Label": "countdownNum"})                
                countdown_num = epoch_df["countdownNum"].iloc[0] if "countdownNum" in epoch_df.columns else event_label
                epoch_filename = os.path.join(outdir, f"{ID}_EKG_epoch_Flanker_run{run}_{countdown_num}.csv")
                epoch_df.to_csv(epoch_filename, index=False)
                print(f"💾 Saved epoch: {epoch_filename}")

        except Exception as e:
            print(f"❌ Error processing {ID} run {run}: {e}")

print("✅ Processing complete for all subjects and runs.")


In [ ]:
# Then, the epochs_create function will take those events and find a one second baseline 
# (-2000 samples when using a 2000hz sampling rate) and also an epoch duration of 4 seconds (8000 samples).
# data has been already downsampled by 20 which means it should be 100 samples now 



In [ ]:
# look at one example of the output 
# understand these cols: 
# examine these columns 
# ECG_Raw, ECG_Clean, ECG_Rate, ECG_Quality timepoint	EVENT	Index	countdownNum	ECG_Baseline_Corrected



In [ ]:
# keep this as a reference 
# example code
eventIndex = events["onset"]
eventList = [] 

for i in eventIndex: 
    d = df1["Event"][i]
    eventList.append(d)

for epoch_index in epoch4: 
    df[epoch_index] = {} 
    epoch = epochs4[epoch_index]
    ecg_baseline = epoch["hr"].loc[-2000:0].mean() 
    ecg_mean = epoch["hr"].loc[0:8000].mean()
    # store ECG in df 
    df[epoch_index]["hr"] = ecg_mean - ecg_baseline 

df = pd.DataFrame.from_dict(df, orient = "index")
df["Event"] = eventList 
df 


In [ ]:
# original sampling rate was 2000 - but has been downsampled by 20
# sampling rate is 100
# 30 seconds = 100 * 30 = 3000 rows 

# create a basline 1 second before
# then create a mean of the next 30 seconds
# not sure what hr is 

for epoch_index in epoch4: 
    df[epoch_index] = {} 
    epoch = epochs4[epoch_index]
    ecg_baseline = epoch["hr"].loc[-100:0].mean()  # change to 100  
    ecg_mean = epoch["hr"].loc[0:3000].mean() # this should be 30 seconds 
    df[epoch_index]["hr"] = ecg_mean - ecg_baseline 

